# LangChain multi-doc retriever with Quentin Tarantino scripts


### Sources
Video: https://www.youtube.com/watch?v=3yPBVii7Ct0
Adapted from: https://colab.research.google.com/drive/1gyGZn_LZNrYXYXa-pltFExbptIe7DAPe?usp=sharing#scrollTo=XHVE9uFb3Ajj

### Contents


## 1. Setting up


In [1]:
!pip show langchain

Name: langchain
Version: 0.0.136
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/michielbontenbal/anaconda3/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [1]:
import os
import config
os.environ["OPENAI_API_KEY"] = config.openai_key

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

## 2. Load multiple text documents and split into chunks



In [4]:
!find . -type f -name "*.txt"

./My Clippings.txt
./pulp-fiction-1994.txt
./hnswlib/CMakeLists.txt
./reservoir-dogs-1992.txt
./state_of_the_union.txt
./jackie-brown-1997.txt


In [5]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()

In [6]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [7]:
len(texts)

591

In [8]:
texts[3]

Document(page_content='bank. You take more of a risk. Banks are\neasier! Federal banks aren\'t supposed to\nstop you anyway, during a robbery. They\'re\ninsured, why should they care? You don\'t\neven need a gun in a federal bank. I heard\nabout this guy, walked into a federal bank\nwith a portable phone, handed the phone to\nthe teller, the guy on the other end of\nthe phone said: "We got this guy\'s little\ngirl, and if you don\'t give him all your\nmoney, we\'re gonna kill \'er."\nYOUNG WOMAN\nDid it work?}\nYOUNG MAN\nFuckin\' A it worked, that\'s what I\'m\ntalkin\' about! Knucklehead walks in a bank\nwith a telephone, not a pistol, not a\nshotgun, but a fuckin\' phone, cleans the\nplace out, and they don\'t lift a fuckin\'\nfinger.\nYOUNG WOMAN\nDid they hurt the little girl?\nYOUNG MAN\nI don\'t know. There probably never was a\nlittle girl – the point of the story isn\'t\nthe little girl. The point of the story is\nthey robbed the bank with a telephone.\nYOUNG WOMAN\nYou wanna 

## 3. Create the vector database

In [9]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

Using embedded DuckDB with persistence: data will be stored in: db


In [10]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [11]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

Using embedded DuckDB with persistence: data will be stored in: db


## 4. Make a retriever

In [12]:
retriever = vectordb.as_retriever()

In [13]:
docs = retriever.get_relevant_documents("Who is vincent vega?")

In [14]:
len(docs)

4

In [15]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [16]:
retriever.search_type

'similarity'

In [17]:
retriever.search_kwargs

{'k': 2}

## 5. Make a chain

In [18]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [19]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "Who is Vincent Vega?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [24]:
# break it down
query = "Who is Jackie Brown?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'Who is Jackie Brown?',
 'result': " Jackie Brown is a very attractive black woman in her mid forties, though she looks like she's in her mid-thirties. She is a stewardess for the Cabo Air shuttle airline, flying from Los Angeles to Cabo San Lucas.",
 'source_documents': [Document(page_content="Jackie being led into the Admitting Area by TWO SHERIFFS . She's wearing her stewardess\nuniform and carrying a small envelope with her belongings in it and her shoes. When Max was\nimagining a woman in her forties, he had someone with a bit of wear and tear on them in mind.\nBut this Jackie Brown's a knockout.\nAs he watches her, she steps out of the County Jail slippers she was wearing and slips into her\nshoes.\nHe approaches, handing her his card.\nMAX\nMiss Brown... I'm Max Cherry. I'm your bail bondsman.\nShe takes the card and shakes his hand saying nothing.\nMAX (CONT'D)\nI can give you a lift home if you'd like?\nJACKIE\nOkay.\nINT. MAX'S CADILLAC – NIGHT\nMax puts his key in 

In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f9f7dc82aa0>)

In [25]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleteing the DB

In [ ]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma-collections.parquet (deflated 50%)
  adding: db/index/ (stored 0%)
  adding: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 5%)
  adding: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 39%)
  adding: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin (deflated 17%)
  adding: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 35%)
  adding: db/chroma-embeddings.parquet (deflated 29%)


In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

restart the runtime

In [ ]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma-collections.parquet  
   creating: db/index/
  inflating: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin  
  inflating: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/chroma-embeddings.parquet  


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Pando raised $30 million in a Series B round, bringing its total raised to $45 million.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


### Chat prompts

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
